<a href="https://colab.research.google.com/github/ShrijaVarma/fake-news-prediction/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 1: Fake news
### 0: real news

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Dta Pre-Processing

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fake_prediction.csv to fake_prediction.csv


In [ ]:
news_dataset = pd.read_csv('fake_prediction.csv')
news_dataset.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [ ]:
news_dataset.shape

(23196, 5)

In [ ]:
## couning number of missing values in dataset
news_dataset.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [ ]:
## replaced null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the title and source_domain
news_dataset['content'] = news_dataset['title'] + ' ' + news_dataset['source_domain']

In [ ]:
print(news_dataset['content'])

0        Kandi Burruss Explodes Over Rape Accusation on...
1        People's Choice Awards 2018: The best red carp...
2        Sophia Bush Sends Sweet Birthday Message to 'O...
3        Colombian singer Maluma sparks rumours of inap...
4        Gossip Girl 10 Years Later: How Upper East Sid...
                               ...                        
23191    Pippa Middleton wedding: In case you missed it...
23192    Zayn Malik & Gigi Hadid’s Shocking Split: Why ...
23193    Jessica Chastain Recalls the Moment Her Mother...
23194    Tristan Thompson Feels "Dumped" After Khloé Ka...
23195    Kelly Clarkson Performs a Medley of Kendrick L...
Name: content, Length: 23196, dtype: object


In [ ]:
# seperatig the data and label
X = news_dataset.drop(columns='real', axis=1)
Y = news_dataset['real']

In [ ]:
print(X)
print(Y)

                                                   title  ...                                            content
0      Kandi Burruss Explodes Over Rape Accusation on...  ...  Kandi Burruss Explodes Over Rape Accusation on...
1      People's Choice Awards 2018: The best red carp...  ...  People's Choice Awards 2018: The best red carp...
2      Sophia Bush Sends Sweet Birthday Message to 'O...  ...  Sophia Bush Sends Sweet Birthday Message to 'O...
3      Colombian singer Maluma sparks rumours of inap...  ...  Colombian singer Maluma sparks rumours of inap...
4      Gossip Girl 10 Years Later: How Upper East Sid...  ...  Gossip Girl 10 Years Later: How Upper East Sid...
...                                                  ...  ...                                                ...
23191  Pippa Middleton wedding: In case you missed it...  ...  Pippa Middleton wedding: In case you missed it...
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...  ...  Zayn Malik & Gigi Hadid’s Shockin

In [ ]:
## Stemming
## it is process of reducing a word to its root word
## actor,acting --> act
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    # Remove all non-alphabet characters
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    return ' '.join(stemmed_content)

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        kandi burruss explod rape accus real housew at...
1        peopl choic award best red carpet look www tod...
2        sophia bush send sweet birthday messag one tre...
3        colombian singer maluma spark rumour inappropr...
4        gossip girl year later upper east sider shock ...
                               ...                        
23191    pippa middleton wed case miss pippa marri lace...
23192    zayn malik gigi hadid shock split chanc reunit...
23193    jessica chastain recal moment mother boyfriend...
23194    tristan thompson feel dump khlo kardashian ref...
23195    kelli clarkson perform medley kendrick lamar h...
Name: content, Length: 23196, dtype: object


In [ ]:
# seperatig the data and label
X = news_dataset['content'].values
Y = news_dataset['real'].values

In [ ]:
print(X)

['kandi burruss explod rape accus real housew atlanta reunion video toofab com'
 'peopl choic award best red carpet look www today com'
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva www etonlin com'
 ...
 'jessica chastain recal moment mother boyfriend slap kick genit www justjar com'
 'tristan thompson feel dump khlo kardashian refus let move la home exclus www intouchweekli com'
 'kelli clarkson perform medley kendrick lamar humbl hit billboard music award www billboard com']


In [ ]:
print(Y)

[1 1 1 ... 1 0 1]


In [ ]:
Y.shape

(23196,)

In [ ]:
## converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 246429 stored elements and shape (23196, 14435)>
  Coords	Values
  (0, 59)	0.2597097788538541
  (0, 651)	0.30737565657585053
  (0, 1707)	0.38749399933436923
  (0, 2460)	0.0508218665886508
  (0, 4193)	0.35506164978733906
  (0, 5891)	0.26124620267491605
  (0, 6647)	0.38749399933436923
  (0, 10223)	0.302158398468634
  (0, 10276)	0.23128301802698967
  (0, 10569)	0.2543303205934995
  (0, 12980)	0.2946734912504806
  (0, 13681)	0.21108934405813046
  (1, 729)	0.3083771474680274
  (1, 1101)	0.33844294892026705
  (1, 1916)	0.3875096512688879
  (1, 2219)	0.42251489448694257
  (1, 2460)	0.08159478582438673
  (1, 7432)	0.33284454859410584
  (1, 9476)	0.23697414575567807
  (1, 10347)	0.37702288594702094
  (1, 12946)	0.36817044163275126
  (1, 14274)	0.10189043319535993
  (2, 1201)	0.19486226938688445
  (2, 1530)	0.3779018214766305
  (2, 1711)	0.3321987353251551
  :	:
  (23194, 4387)	0.2692478125672807
  (23194, 5800)	0.2632849856372537
  (

In [50]:
## Splitting the dataset to training & test data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [54]:
##Training the model: Logistic Regression



In [60]:
print(X_train.shape)
print(Y_train.shape)

(18556, 14435)
(18556,)


In [61]:
model = LogisticRegression()
model.fit(X_train, Y_train)


LogisticRegression()

In [63]:
## accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [64]:
print('Accuracy score of training data: ', training_data_accuracy)

Accuracy score of training data:  0.8821944384565639


In [65]:
## accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [66]:
print('Accuracy score of test data: ', test_data_accuracy)

Accuracy score of test data:  0.8543103448275862


In [68]:
## Making predictive System
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
  print('The news is Real')
else:
    print('The news is Fake')

[1]
The news is Fake
